## Import Statements

In [1]:
import pandas as pd
import yfinance as yf
import os
import datetime as dt

## More simplistic and fantastic way to get data table inside Web HTML as dataframes

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
constituents_table = payload[0]
df = constituents_table

## Output Structure of Dataframe from web table

In [3]:
df.head(5)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
index_symbols = df['Symbol']
total_symbols = len(index_symbols)

## Accumulate all ticker symbols to get Ticker Object from YF in one call

In [5]:
all_tickers = "";
for index, symbol in index_symbols.iteritems(): 
    new_symbol = symbol.replace(".","-")
    all_tickers += new_symbol + ' ';
print(all_tickers)

MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AAPL AMAT APTV ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BDX BRK-B BBY BIO BIIB BLK BA BKNG BWA BXP BSX BMY AVGO BR BF-B CHRW COG CDNS CZR CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE CNC CNP CERN CF SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ COO CPRT GLW CTVA COST CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DISCA DISCK DISH DG DLTR D DPZ DOV DOW DTE DUK DRE DD DXC EMN ETN EBAY ECL EIX EW EA EMR ENPH ETR EOG EFX EQIX EQR ESS EL ETSY EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GNRC GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HIG HAS HCA PEAK HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HWM HPQ HUM HBAN H

In [13]:
all_tickers = 'MXIM'
stock_data = yf.Tickers(all_tickers)

## Sample data read from ticker object

In [14]:
stock_data.tickers[0].history(period="1").head(1)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-03-24,90.05,91.46,89.38,89.39,1900802,0,0


In [15]:
symbols_data = []
sec_name_data = []
sector_data = []
industry_data = []
close_price_data = []
previous_close_price_data = []
floating_shares_data = []

## Core logic to iterate all index constituents and get data points

##### Note: Replace dot in ticker with hypen for yahoo finance api to work correctly

In [16]:
for index, symbol in index_symbols.iteritems(): 
    try:
        modified_symbol = symbol.replace('.','-')
        print('Processing stock#', index)
        stock_info = stock_data.tickers[index].info
        data = stock_data.tickers[index].history(period="1d")
        symbols_data.append(symbol);
        sec_name_data.append(df[df['Symbol'] == symbol]['Security'].values[0]);
        
        sector_data.append(df[df['Symbol'] == symbol]['GICS Sector'].values[0]);
        industry_data.append(df[df['Symbol'] == symbol]['GICS Sub-Industry'].values[0]);

        close_price_data.append(data.iloc[0]['Close'])

        if('previousClose' in stock_info):
            previous_close_price_data.append(stock_info['previousClose'])
        else:
            print('PREVIOUS CLOSE NOT FOUND FOR SYMBOL ', symbol)
            previous_close_price_data.append(None)

        if('floatShares' in stock_info):
            floating_shares_data.append(stock_info['floatShares'])
        else:
            print('FLOATING SHARES NOT FOUND FOR SYMBOL ', symbol)
            floating_shares_data.append(None)
    except:
        print('ERROR PROCESSING SYMBOL ', symbol)


Processing stock# 0
Processing stock# 1
ERROR PROCESSING SYMBOL  ABT
Processing stock# 2
ERROR PROCESSING SYMBOL  ABBV
Processing stock# 3
ERROR PROCESSING SYMBOL  ABMD
Processing stock# 4
ERROR PROCESSING SYMBOL  ACN
Processing stock# 5
ERROR PROCESSING SYMBOL  ATVI
Processing stock# 6
ERROR PROCESSING SYMBOL  ADBE
Processing stock# 7
ERROR PROCESSING SYMBOL  AMD
Processing stock# 8
ERROR PROCESSING SYMBOL  AAP
Processing stock# 9
ERROR PROCESSING SYMBOL  AES
Processing stock# 10
ERROR PROCESSING SYMBOL  AFL
Processing stock# 11
ERROR PROCESSING SYMBOL  A
Processing stock# 12
ERROR PROCESSING SYMBOL  APD
Processing stock# 13
ERROR PROCESSING SYMBOL  AKAM
Processing stock# 14
ERROR PROCESSING SYMBOL  ALK
Processing stock# 15
ERROR PROCESSING SYMBOL  ALB
Processing stock# 16
ERROR PROCESSING SYMBOL  ARE
Processing stock# 17
ERROR PROCESSING SYMBOL  ALXN
Processing stock# 18
ERROR PROCESSING SYMBOL  ALGN
Processing stock# 19
ERROR PROCESSING SYMBOL  ALLE
Processing stock# 20
ERROR PROCES

### Populate all collected data into dataframe - TODO: Need to find a better way to do it

In [17]:
out_data = {'Symbol': symbols_data,
            'Security': sec_name_data,
            'Sector': sector_data,
            'Industry': industry_data,
            'Close_Price': close_price_data,
            'Previous_Day_Close_Price': previous_close_price_data, 
            'Floating_Shares': floating_shares_data}
output_df = pd.DataFrame(out_data, columns= ['Symbol', 'Security', 'Sector', 'Industry','Close_Price','Previous_Day_Close_Price', 'Floating_Shares'])

## Sample Output Data

In [19]:
output_df.head(100)

,Symbol,Security,Sector,Industry,Close_Price,Previous_Day_Close_Price,Floating_Shares
0,MMM,3M Company,Industrials,Industrial Conglomerates,89.39,88.85,266170140


## Writing output data to excel file for further analysis

In [12]:
output_dir = '../results/sp500'
if not os.path.exists(output_dir):
    os.mkdir(output_dir);
    
today_date = dt.datetime.today().strftime('%Y-%m-%d')
output_file = 'performance_analysis_'+today_date+'.xlsx';

file_path = os.path.join(output_dir, output_file)

if os.path.exists(file_path):
    os.remove(file_path)

output_df.to_excel(file_path)